In [7]:
import json
# import pandas as pd
import requests
# from tqdm import tqdm
# import numpy as np

# index = np.arange(5)

# test_data = pd.DataFrame([{'name':'제주시청','address':'제주시 이도이동 1176-1'},
#                   {'name':'함덕해수욕장','address':'제주시 조천읍 함덕리 1004-10'},
#                   {'name':'협재해수욕장','address':'제주시 한림읍 협재리 2497-1'},
#                   {'name':'쇠소깍','address':'서귀포시 남원읍 하례리 995-1'},
#                   {'name':'성산일출봉','address':'서귀포시 성산읍 성산리 1'}],index=index)

url = 'http://api.vworld.kr/req/address?'
# params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=false&simple=false&format=json&type='
# road_type = 'ROAD'
# address = '&address='
# keys = '&key='
# primary_key = '4A1CF381-0E54-33E4-AC30-0357FFAE3951'
param1 = 'service=address'
param2 = 'request=getCoord'
param3 = 'key=4A1CF381-0E54-33E4-AC30-0357FFAE3951'
param4 = 'type=road'
param5 = 'refine=false'
param6 = 'address=부림로169번길 22'

def request_geo(road):
    link = 'http://api.vworld.kr/req/address?{0}&{1}&{2}&{3}&{4}&{5}'.format(param1,param2,param3,param4,param5,param6)
    page = requests.get(link)
    json_data = page.json()
    
    status = json_data['response']['status']
    x = json_data['response']['result']['point']['x']
    y = json_data['response']['result']['point']['y']
    print('status :')
    

# def extraction_geo(test_data):
#     geocode = pd.DataFrame(columns = ['name','address', 'x', 'y'])
#     none = None
#     for idx, road in tqdm(zip(test_data.index ,test_data['address'])):
#         name = str(test_data['name'][idx])
#         if len(str(road)) <= 5:
#             geocode = geocode.append(
#                     pd.DataFrame({'name':name,
#                     'address':road,
#                     'x':none,
#                     'y':none},
#                     index=[idx]))
#             continue

#         json_data = request_geo(road)

#         if json_data['response']['status'] == 'NOT_FOUND' or json_data['response']['status'] == 'ERROR':
#             geocode = geocode.append(
#                     pd.DataFrame({'name':name,
#                     'address':road,
#                     'x':none,
#                     'y':none},
#                     index=[idx]))
#             continue

#         x = json_data['response']['result']['point']['x']
#         y = json_data['response']['result']['point']['y']

#         geocode = geocode.append(
#             pd.DataFrame({'name':name,
#                     'address':road,
#                     'x':float(x),
#                     'y':float(y)},
#                     index=[idx]))
#     return geocode

# result = extraction_geo(test_data)
print(request_geo('부림로 169번길22'))

{'response': {'service': {'name': 'address', 'version': '2.0', 'operation': 'getCoord', 'time': '35(ms)'}, 'status': 'OK', 'input': {'type': 'road', 'address': '부림로169번길 22'}, 'result': {'crs': 'EPSG:4326', 'point': {'x': '126.960877639', 'y': '37.398095842'}}}}
